# We read model:

In [1]:
# First, we can import some functions so we can use the model
from cobra.io import read_sbml_model, write_sbml_model
from cobra import Reaction, Metabolite

# Second, we can read the GEM and save it as ‘model’
model = read_sbml_model('data/iML1515.xml')

# Third, we can show general information about the loaded model
model

Name,iML1515
Memory address,1b226d42790
Number of metabolites,1877
Number of reactions,2712
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## We frist check for the needed reactions and metabolites already existing in the model:

### The pathway:

ac_c <--ACKr--> actp_c <--PTAr--> accoa_c <--ACACT1r--> aacoa_c <--HACD1--> 3hbcoa_c --phaC--> P3HB

In [2]:
for metabolite in model.metabolites.query('ac_c', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Acetate ac_c C2H3O2
Phenylacetic acid pac_c C8H7O2
Sulfoacetate sulfac_c C2H2O5S
Isethionic acid isetac_c C2H5O4S
Acetoacetate acac_c C4H5O3
3-Aminoacrylate 3amac_c C3H4NO2
Nicotinate nac_c C6H4NO2
Peroxyaminoacrylate poaac_c C3H5NO3


In [3]:
for reaction in model.reactions.query('ACKr', 'id'):
    print(reaction.name, reaction)

Acetate kinase ACKr: ac_c + atp_c <=> actp_c + adp_c


In [4]:
for metabolite in model.metabolites.query('actp_c', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Acetyl phosphate actp_c C2H3O5P


In [5]:
for reaction in model.reactions.query('PTAr', 'id'):
    print(reaction.name, reaction)

Phosphotransacetylase PTAr: accoa_c + pi_c <=> actp_c + coa_c


In [6]:
for metabolite in model.metabolites.query('accoa_c', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Phenylacetyl-CoA phaccoa_c C29H38N7O17P3S
Acetyl-CoA accoa_c C23H34N7O17P3S
2-oxepin-2(3H)-ylideneacetyl-CoA 2oxpaccoa_c C29H38N7O18P3S
Ring 1,2-epoxyphenylacetyl-CoA rephaccoa_c C29H38N7O18P3S


In [7]:
for reaction in model.reactions.query('ACACT1r', 'id'):
    print(reaction.name, reaction)

Acetyl-CoA C-acetyltransferase ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c


In [8]:
for metabolite in model.metabolites.query('aacoa_c', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Acetoacetyl-CoA aacoa_c C25H36N7O18P3S


In [9]:
for reaction in model.reactions.query('HACD1', 'id'):
    print(reaction.name, reaction)

3-hydroxyacyl-CoA dehydrogenase (acetoacetyl-CoA) HACD1: aacoa_c + h_c + nadh_c <=> 3hbcoa_c + nad_c


In [10]:
for metabolite in model.metabolites.query('3hbcoa_c', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

(S)-3-Hydroxybutanoyl-CoA 3hbcoa_c C25H38N7O18P3S


In [11]:
for reaction in model.reactions.query('phaC', 'id'):
    print(reaction.name, reaction)

Not Found

In [14]:
for metabolite in model.metabolites.query('P3HB', 'id'):
    print(metabolite.name, metabolite.id, metabolite.formula)

Not Found

## We add the missing reaction (phaC) and metabolite (P3HB):

In [15]:
# phaC reaction

new_reaction2 = Reaction('phaC', '3-Hydroxybutyryl-CoA: P3HB polymerization')  # The reaction corresponding to phaC

# phaC uses:
## 10*3-Hydroxybutyryl-CoA <=> P3HB + 10*CoA
## We can find these as: (Note: All of these are in the cytoplasm)
### 3-Hydroxybutyryl-CoA = 3hbcoa_c

### P3HB                 = Not Defined yet
### CoA                  = coa_c


# Since Poly 3-hydroxibutanoate does not exist in the model, we will have to define it before.
# Since it is a polymer of formula [C4H6O2]n and we cannot write that like this in the model, we'll assume n = 10.
P3HB = Metabolite(id='P3HB_c', name='poly-3-Hydroxybutanoate', compartment='c', formula='C40H62O21')


# We can now define the reaction stoichiometry:
new_reaction2.add_metabolites({
                            model.metabolites.get_by_id("3hbcoa_c"): -10,
                            P3HB: 1,
                            model.metabolites.get_by_id("coa_c"): 10
                             })

model.add_reactions([new_reaction2])

In [16]:
model.reactions.phaC

Reaction identifier,phaC
Name,3-Hydroxybutyryl-CoA: P3HB polymerization
Memory address,0x1b238bbab90
Stoichiometry,10 3hbcoa_c --> P3HB_c + 10 coa_c 10 (S)-3-Hydroxybutanoyl-CoA --> poly-3-Hydroxybutanoate + 10 Coenzyme A
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
model.metabolites.P3HB_c

Metabolite identifier,P3HB_c
Name,poly-3-Hydroxybutanoate
Memory address,0x1b238b7d150
Formula,C40H62O21
Compartment,c
In 1 reaction(s),phaC


## Let's do some testing:

In [18]:
# Here, we are testing if the intermediate compound can be produced

with model:
    r_phaA_exp = model.add_boundary(model.metabolites.get_by_id('aacoa_c'), type='demand')
    model.objective = r_phaA_exp
    sol = model.optimize()
    print(sol.objective_value)

1.789420654911844


In [19]:
# Here, we are testing if the intermediate compound can be produced

with model:
    r_phaB_exp = model.add_boundary(model.metabolites.get_by_id('3hbcoa_c'), type='demand')
    model.objective = r_phaB_exp
    sol = model.optimize()
    print(sol.objective_value)

1.7671641791044876


In [20]:
# Here, we are testing if the intermediate compound can be accomulated

with model:
    r_phaC_exp = model.add_boundary(model.metabolites.get_by_id('P3HB_c'), type='sink')
    model.objective = r_phaC_exp
    sol = model.optimize()
    print(sol.objective_value)

1.2500530973451296


## Let's save the new model:

In [22]:
write_sbml_model(model, "data/iML1515_het.xml")

In [23]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.004565,0,0.00%
cl_e,EX_cl_e,0.004565,0,0.00%
cobalt2_e,EX_cobalt2_e,2.192E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006218,0,0.00%
fe2_e,EX_fe2_e,0.01409,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1712,0,0.00%
mg2_e,EX_mg2_e,0.007608,0,0.00%
mn2_e,EX_mn2_e,0.000606,0,0.00%
mobd_e,EX_mobd_e,6.139E-06,0,0.00%
